In [69]:
import torch
if torch.backends.mps.is_available():
    mps_device = torch.device("mps")
    x = torch.ones(1, device=mps_device)
    print (x)
else:
    print ("MPS device not found.")

MPS device not found.


In [70]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import matplotlib.pyplot as plt

from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision import transforms

device = torch.device('mps' if torch.backends.mps.is_available() else 'cpu')

transforms = transforms.Compose([transforms.ToTensor()])

train_dataset = datasets.MNIST('/tmp/data', train=True, download=True, 
                               transform=transforms)

test_dataset = datasets.MNIST('./data', train=False, download=True,
                              transform=transforms)

print(torch.backends.mps.is_built())
print(torch.backends.mps.is_available())
device

False
False


device(type='cpu')

In [71]:
BATCH_SIZE      = 64        # number of data points in each batch
N_EPOCHS        = 10        # times to run the model on complete data
INPUT_DIM       = 28 * 28   # size of each input
HIDDEN_DIM      = 256       # hidden dimension
LATENT_DIM      = 20        # latent vector dimension
lr              = 1e-3      # learning rate

In [72]:
train_iterator = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
test_iterator = DataLoader(test_dataset, batch_size=BATCH_SIZE)

In [73]:
class Encoder(nn.Module):
  ''' This the encoder part of VAE'''

  def __init__(self, input_dim: int, hidden_dim: int, z_dim: int):
    '''
    Args:
      input_dim: A integer indicating the size of input 
        (in case of MNIST 28 * 28).
      hidden_dim: A integer indicating the size of hidden dimension.
      z_dim: A integer indicating the latent dimension.
    '''
    super(Encoder, self).__init__()

    self.z_dim = z_dim
    self.encoder = nn.Sequential(
      nn.Linear(input_dim, hidden_dim), 
      nn.ReLU(),
      nn.Linear(hidden_dim, hidden_dim), 
      nn.ReLU(),
      nn.Linear(hidden_dim, 2*z_dim),
    )

  def forward(self, x: torch.Tensor):
    # x is of shape [batch_size, input_dim]

    hidden = self.encoder(x)
    z_mu, z_logvar = hidden[:, :self.z_dim], hidden[:, self.z_dim:]

    return z_mu, z_logvar

In [74]:
class Decoder(nn.Module):
  ''' This the decoder part of VAE'''

  def __init__(self, z_dim: int, hidden_dim: int, output_dim: int):
    '''
    Args:
        z_dim: A integer indicating the latent size.
        hidden_dim: A integer indicating the size of hidden dimension.
        output_dim: A integer indicating the output dimension 
          (in case of MNIST it is 28 * 28)
    '''
    super(Decoder, self).__init__()

    self.decoder = nn.Sequential(
      nn.Linear(z_dim, hidden_dim), 
      nn.ReLU(),
      nn.Linear(hidden_dim, hidden_dim), 
      nn.ReLU(),
      nn.Linear(hidden_dim, output_dim),
      nn.Sigmoid()
    )

  def forward(self, x: torch.Tensor):
    return self.decoder(x)

# Variational AutoEncoder

\begin{align}
\boldsymbol{\mu}_x, \boldsymbol{\sigma}_x &= M(\textbf{x}), \Sigma(\textbf{x}) && \text{Push $\textbf{x}$ through encoder}
\\ \\
\boldsymbol{\epsilon} &\sim \mathcal{N}(0, 1) && \text{Sample noise}
\\ \\
\textbf{z} &= \boldsymbol{\epsilon} \boldsymbol{\sigma}_x + \boldsymbol{\mu}_x  && \text{Reparameterize}
\\ \\
\textbf{x}_r &= p_{\boldsymbol{\theta}}(\textbf{x} \mid \textbf{z}) && \text{Push $\textbf{z}$ through decoder}
\end{align}

In [75]:
class VAE(nn.Module):
  ''' This the VAE, which takes a encoder and decoder.'''

  def __init__(self, enc: Encoder, dec: Decoder):
    super(VAE, self).__init__()
    self.enc = enc
    self.dec = dec

  def reparameterization_trick(self, z_mu: torch.Tensor, 
                                z_logvar: torch.Tensor):
  
    # implement reparametrization trick
    return z_mu + torch.exp(z_logvar) * torch.randn_like(z_mu)

  def forward(self, x: torch.Tensor):
    
    # encode
    z_mu, z_logvar = self.enc(x)

    # sample z from posterior distribution
    z_post = self.reparameterization_trick(z_mu, z_logvar)
    
    # decode
    predicted = self.dec(z_post)

    return predicted, z_mu, z_logvar

  def sample(self, num_samples: int = 1):
    
    # implement sampling of num_samples datapoint
    z = torch.randn((num_samples, LATENT_DIM), device=device)
    return self.dec(z)

# ELBO computation

\begin{align}
\text{recon. loss} &= \text{MSE}(\textbf{x}, \textbf{x}_r) \ \ \text{or} \ \  \text{BCE}(\textbf{x}, \textbf{x}_r) && \text{Compute reconstruction loss}
\\ \\
\text{var. loss} &= \text{KL}[\mathcal{N}(\boldsymbol{\mu}_x, \boldsymbol{\sigma}_x) \lVert \mathcal{N}(0, I)] && \text{Compute variational loss}
\\ \\
\text{L} &= \text{recon. loss} + \text{var. loss} && \text{Combine losses}
\end{align}

\\

\begin{aligned}
\text{KL}[\mathcal{N}(\mu_1, \Sigma_1) \lVert \mathcal{N}(\mu_2, \Sigma_2))] &= \\ 
& \frac{1}{2}\left[\log\frac{|\Sigma_2|}{|\Sigma_1|} - d + \text{tr} \{ \Sigma_2^{-1}\Sigma_1 \} + (\mu_2 - \mu_1)^T \Sigma_2^{-1}(\mu_2 - \mu_1)\right], \text{where}~~d~~\text{is the number of dimensions}
\end{aligned}

In [76]:
class ELBO(nn.Module):

  def __init__(self):
    super(ELBO, self).__init__()

  def compute_rec_error_(self, x: torch.Tensor, x_rec: torch.Tensor):
    # implement reconstruction error
    return F.binary_cross_entropy(x_rec, x, reduction='sum')
  
  def compute_kl_(sefl, z_mu: torch.Tensor, z_logvar: torch.Tensor):
    # implement kl divergence (see above, think of what you're matching)
    return 0.5 * torch.sum(torch.exp(z_logvar) + z_mu.pow(2) - - 1.0 - z_logvar)

  def forward(self, x: torch.Tensor, x_rec: torch.Tensor, 
              z_mu: torch.Tensor, z_logvar: torch.Tensor): 

    # reconstruction loss
    recon_loss = self.compute_rec_error_(x, x_rec)

    # kl divergence loss
    kl_loss = self.compute_kl_(z_mu, z_logvar)

    # total loss
    loss = recon_loss + kl_loss

    return loss

In [77]:
from torch.utils.data import DataLoader

def eval(data_loader: DataLoader, model: VAE):
    
  # set the evaluation mode
  model.eval()

  # test loss for the data
  test_loss = 0
  num_examples = 0

  # define the ELBO loss function
  loss = ELBO()

  # we don't need to track the gradients, since we are not updating the 
  # parameters during evaluation / testing
  with torch.no_grad():
    for i, (x, _) in enumerate(data_loader):
      # reshape the data
      x = x.view(-1, 28 * 28)
      x = x.to(device)

      # forward pass
      x_rec, z_mu, z_logvar = model(x)

      test_loss += loss(x, x_rec, z_mu, z_logvar).item()
      num_examples += len(x)

  return test_loss/num_examples

In [78]:
encoder   = Encoder(INPUT_DIM, HIDDEN_DIM, LATENT_DIM) # encoder
decoder   = Decoder(LATENT_DIM, HIDDEN_DIM, INPUT_DIM) # decoder
model     = VAE(encoder, decoder).to(device) # vae
optimizer = optim.Adam(model.parameters(), lr=lr)  # optizer
loss_fun  = ELBO()

for e in range(N_EPOCHS):

  # set the train mode
  model.train()
  
  for i, (x, _) in enumerate(train_iterator):
      
    # reshape the data into [batch_size, 784]
    x = x.view(-1, 28 * 28).to(device)

    # update the gradients to zero
    optimizer.zero_grad()

    # forward pass
    x_rec, z_mu, z_logvar = model(x)

    loss = loss_fun(x, x_rec, z_mu, z_logvar)
    
    # backward pass
    loss.backward()
    
    # update the weights
    optimizer.step()

  print(f'Epoch {e}, Train Loss: {eval(train_iterator, model):.2f}, \
        Test Loss: {eval(test_iterator, model):.2f}')

Epoch 0, Train Loss: 126.31,         Test Loss: 125.45
Epoch 1, Train Loss: 109.02,         Test Loss: 108.14
Epoch 2, Train Loss: 102.47,         Test Loss: 101.76
Epoch 3, Train Loss: 99.64,         Test Loss: 99.18
Epoch 4, Train Loss: 97.27,         Test Loss: 96.97
Epoch 5, Train Loss: 96.34,         Test Loss: 96.30
Epoch 6, Train Loss: 95.13,         Test Loss: 95.18
Epoch 7, Train Loss: 94.08,         Test Loss: 94.26
Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "/opt/anaconda3/envs/MLDL-Lab/lib/python3.8/site-packages/IPython/core/interactiveshell.py", line 3508, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/var/folders/rc/973wdqd91sq6bj8vj_ylcf900000gn/T/ipykernel_18701/367033775.py", line 29, in <module>
    optimizer.step()
  File "/opt/anaconda3/envs/MLDL-Lab/lib/python3.8/site-packages/torch/optim/optimizer.py", line 391, in wrapper
    if "_optimizer_load_state_dict_pre_hooks" not in self.__dict__:
  File "/opt/anaconda3/envs/MLDL-Lab/lib/python3.8/site-packages/torch/optim/optimizer.py", line 76, in _use_grad
    # we need to graph break to ensure that aot respects the no_grad annotation.
  File "/opt/anaconda3/envs/MLDL-Lab/lib/python3.8/site-packages/torch/optim/adam.py", line 168, in step
  File "/opt/anaconda3/envs/MLDL-Lab/lib/python3.8/site-packages/torch/optim/adam.py", line 318, in adam
    )
  File "/opt/anaconda3/envs/MLDL-Lab/lib/python3.8/site-packages/t

# Example: sampling

In [12]:
img = model.sample()
img = img.view(28, 28)
img = img.cpu().detach().numpy()

plt.imshow(img, cmap='gray')

RuntimeError: Placeholder storage has not been allocated on MPS device!

# Example: interpolation

In [ ]:
number_one = train_dataset.data[train_dataset.targets == 1][0]
number_five = train_dataset.data[train_dataset.targets == 5][0]

with torch.no_grad():
  one_mu, one_logvar = model.enc((number_one.view(-1, 28 * 28) / 255.).to(device))
  z_one = model.reparameterization_trick(one_mu, one_logvar)
  five_mu, five_logvar = model.enc((number_five.view(-1, 28 * 28) / 255.).to(device))
  z_five = model.reparameterization_trick(five_mu, five_logvar)

  fig, ax = plt.subplots(1, 7, figsize=(21,3))
  for n, i in enumerate(torch.linspace(0, 1, 7)):
    predicted = model.dec(z_five * i + z_one * (1-i)).cpu()
    ax[n].imshow(predicted.view(28,28))